In [1]:
%load_ext autoreload
%autoreload 2
from config import MARKET_MODEL_DIR,MARKET_MODEL_DIR_TESTING
from model_factory import build_market_neutral_model, build_market_neutral_model_testing
import pickle
with open(MARKET_MODEL_DIR, "rb") as f:
    params = pickle.load(f)

with open(MARKET_MODEL_DIR_TESTING, "rb") as f:
    params_testing = pickle.load(f)
    
import os
#个人许可证
#os.environ['GRB_LICENSE_FILE'] = os.path.expanduser("~/gurobi/gurobi.lic")


# ## 系统Gurobi
os.environ['GUROBI_HOME'] = '/usr/licensed/gurobi/12.0.0/linux64'
os.environ['GRB_LICENSE_FILE'] = '/usr/licensed/gurobi/license/gurobi.lic'

# 清除个人WLS许可证
for var in ['WLSACCESSID', 'WLSSECRET']:
    if var in os.environ:
        del os.environ[var]

model = build_market_neutral_model(**params)
model_testing= build_market_neutral_model_testing(**params_testing)

Using device: cuda
GPU: NVIDIA A100 80GB PCIe MIG 1g.10gb
Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"


In [2]:
print( params_testing)

{'N': 7, 'A': array([[1., 1., 1., 1., 1., 1., 1.]]), 'b': array([0.]), 'l': array([0., 0., 0., 0., 0., 0., 0.]), 'u': array([1000000., 1000000., 1000000., 1000000., 1000000., 1000000.,
       1000000.]), 'risk_f': array([0.35320865, 0.42288222, 0.37707359, 0.35261655, 0.41933761,
       0.32860565, 0.38218537]), 'risk_abs': 1.5, 'single_abs': 0.1, 'l1_abs': 1.0, 'cov_matrix': array([[4.67496276e-05, 4.15905709e-05, 3.60928100e-05, 3.48342604e-05,
        4.16919822e-05, 3.34891585e-05, 3.66772287e-05],
       [4.15905709e-05, 5.97804688e-05, 4.37975936e-05, 4.19857337e-05,
        5.10524222e-05, 3.90648444e-05, 4.49568886e-05],
       [3.60928100e-05, 4.37975936e-05, 5.26604548e-05, 3.92702956e-05,
        4.44118365e-05, 3.45134604e-05, 3.79007596e-05],
       [3.48342604e-05, 4.19857337e-05, 3.92702956e-05, 4.25771017e-05,
        4.21567423e-05, 3.38763419e-05, 3.56807453e-05],
       [4.16919822e-05, 5.10524222e-05, 4.44118365e-05, 4.21567423e-05,
        5.69786874e-05, 3.9319408

In [3]:
from io_utils import load_dataset_dict, create_dataset_from_dict
from config import DATASET_DICT_PATH, TEST_DATASET_DICT_PATH
dataset_dict = load_dataset_dict(DATASET_DICT_PATH)
dataset_train = create_dataset_from_dict(dataset_dict, model)
test_dataset_dict = load_dataset_dict(TEST_DATASET_DICT_PATH)
dataset_test = create_dataset_from_dict(test_dataset_dict, model)

>>> Precomputed mode enabled. Skipping time series processing and solution computation.
>>> Precomputed mode enabled. Skipping time series processing and solution computation.


# 可选模型的信息
#  model_info = {
        'TCN': {
            # 训练配置
            'use_mixed_precision': True,
            'gradient_clip_norm': None,
            'training_reason': 'TCN架构稳定，无需梯度裁剪。混合精度加速训练。',
            
            # 推荐条件
            'recommend_when': [
                ('sequence_length > 200', '长序列，TCN处理效率最高'),
                ('50 <= sequence_length <= 200', '中等长度序列的最佳选择'),
                ('computational_budget == "medium"', '平衡性能和资源消耗'),
                ('prefer_speed == True', '训练速度快，并行化好'),
            ],
            
            # 适用场景
            'best_for': [
                '长时间序列预测',
                '实时推理场景',
                '需要快速训练的项目',
                'GPU资源充足的环境'
            ],
            
            # 优势
            'advantages': [
                '✅ 梯度稳定，几乎不会爆炸/消失',
                '✅ 并行训练，速度快',
                '✅ 感受野可控，能捕获长期依赖',
                '✅ 架构成熟，调参简单'
            ],
            
            # 劣势
            'disadvantages': [
                '⚠️ 参数量相对较多',
                '⚠️ 对短序列可能过拟合'
            ]
        },
        
        'DLinear': {
            # 训练配置
            'use_mixed_precision': gpu_memory_gb < 8 or prefer_speed,
            'gradient_clip_norm': None,
            'training_reason': 'DLinear极简稳定。小GPU推荐混合精度节省内存。',
            
            # 推荐条件
            'recommend_when': [
                ('has_trend and has_seasonality', '数据有明显的趋势和季节性模式'),
                ('sequence_length < 50', '短序列，简单模型可能更有效'),
                ('computational_budget == "low"', '计算资源有限'),
                ('prefer_speed == True', '极快的训练和推理速度'),
            ],
            
            # 适用场景
            'best_for': [
                '有明显趋势的时间序列',
                '计算资源受限的环境',
                '需要可解释性的场景',
                '作为强基线模型'
            ],
            
            # 优势
            'advantages': [
                '✅ 极简架构，几乎不会过拟合',
                '✅ 训练超快，参数量少',
                '✅ 对趋势和季节性敏感',
                '✅ 可解释性强',
                '✅ 内存占用极少'
            ],
            
            # 劣势
            'disadvantages': [
                '⚠️ 对复杂非线性模式建模能力有限',
                '⚠️ 可能在复杂数据上表现不佳'
            ]
        },
        
        'Transformer': {
            # 训练配置
            'use_mixed_precision': True,
            'gradient_clip_norm': 1.0,
            'training_reason': 'Transformer计算密集，强烈推荐混合精度。轻微梯度裁剪保险。',
            
            # 推荐条件
            'recommend_when': [
                ('computational_budget == "high" and sequence_length < 200', '充足资源且序列不太长'),
                ('need_attention == True', '需要注意力机制和解释性'),
                ('complex_patterns == True', '数据有复杂的长期依赖关系'),
            ],
            
            # 适用场景
            'best_for': [
                '需要注意力可视化的场景',
                '复杂的多变量时间序列',
                '研究和实验项目',
                '对精度要求极高的场景'
            ],
            
            # 优势
            'advantages': [
                '✅ 强大的序列建模能力',
                '✅ 注意力机制提供解释性',
                '✅ 能捕获复杂的长期依赖',
                '✅ 在复杂数据上表现优秀'
            ],
            
            # 劣势
            'disadvantages': [
                '⚠️ 计算复杂度O(n²)',
                '⚠️ 参数量大，容易过拟合',
                '⚠️ 对长序列内存消耗大',
                '⚠️ 需要更多调参'
            ]
        },
        
        'CNN-Attention': {
            # 训练配置
            'use_mixed_precision': True,
            'gradient_clip_norm': 0.5,
            'training_reason': '混合架构，推荐保守的梯度裁剪配置。',
            
            # 推荐条件
            'recommend_when': [
                ('need_local_and_global == True', '需要同时捕获局部和全局特征'),
                ('computational_budget == "medium"', '中等计算资源'),
                ('experimental == True', '实验性尝试'),
            ],
            
            # 适用场景
            'best_for': [
                '局部和全局特征都重要的数据',
                '实验和研究项目',
                '需要平衡性能的场景'
            ],
            
            # 优势
            'advantages': [
                '✅ 结合CNN和Attention优势',
                '✅ 对局部模式敏感',
                '✅ 有一定的全局建模能力'
            ],
            
            # 劣势
            'disadvantages': [
                '⚠️ 架构复杂，调参困难',
                '⚠️ 可能不如专门的模型',
                '⚠️ 训练稳定性一般'
            ]
        }
    }

In [4]:
%load_ext autoreload
%autoreload 2
from config import K, HIDDEN_DIM, LSTM_HIDDEN_DIM, DROPOUT_RATE, DEVICE
from architecture import TwoLayerLSTM, TCNModel, DLinear, LightTransformer,CNNAttentionModel
from seed import set_seed

set_seed(42)


lstm = TwoLayerLSTM(k=K, hidden_dim=HIDDEN_DIM, lstm_hidden_dim=LSTM_HIDDEN_DIM, dropout_rate=DROPOUT_RATE).to(DEVICE)
print("Model architecture:")
for name, param in lstm.named_parameters():
    print(f"{name:30s} | Shape: {param.data.shape} | Parameters: {param.numel()} | require_grad: {param.requires_grad}")
    print(f"Mean Value: {param.data.mean()} | Grad: {param.grad}")
    print("-" * 80)
    
"""
    

#长序列，TCN处理效率最高
lstm = TCNModel(k=K, hidden_dim=HIDDEN_DIM, num_layers=4, dropout_rate=DROPOUT_RATE).to(DEVICE)


# 选项2：DLinear (数据有规律模式时)
set_seed(42)
lstm = DLinear(k=K, lookback=LOOKBACK, individual=True).to(DEVICE)

# 选项3：Transformer (中等长度序列)
set_seed(42)
lstm = LightTransformer(k=K, d_model=HIDDEN_DIM, nhead=4, num_layers=2, dropout_rate=DROPOUT_RATE).to(DEVICE)


# 选项4:CNN-Attention
lstm = CNNAttentionModel(
        k=K, 
        hidden_dim=HIDDEN_DIM, 
        num_heads=4,            # 注意力头数
        dropout_rate=DROPOUT_RATE
    ).to(DEVICE)
"""
print("Model architecture:")
for name, param in lstm.named_parameters():
    print(f"{name:30s} | Shape: {param.data.shape} | Parameters: {param.numel()} | require_grad: {param.requires_grad}")
    print(f"Mean Value: {param.data.mean()} | Grad: {param.grad}")
    print("-" * 80)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Seed set to 42
Model architecture:
lstm.weight_ih_l0              | Shape: torch.Size([256, 21]) | Parameters: 5376 | require_grad: True
Mean Value: -0.00018090897356159985 | Grad: None
--------------------------------------------------------------------------------
lstm.weight_hh_l0              | Shape: torch.Size([256, 64]) | Parameters: 16384 | require_grad: True
Mean Value: -0.0004572543257381767 | Grad: None
--------------------------------------------------------------------------------
lstm.bias_ih_l0                | Shape: torch.Size([256]) | Parameters: 256 | require_grad: True
Mean Value: 0.006879021879285574 | Grad: None
--------------------------------------------------------------------------------
lstm.bias_hh_l0                | Shape: torch.Size([256]) | Parameters: 256 | require_grad: True
Mean Value: 0.006068169604986906 | Grad: None
----------------------------------------------

In [5]:
from data_loader import get_data_loaders
# 创建 Subset 类
from torch.utils.data import DataLoader
from config import BATCH_SIZE
# 假设 dataset_train, dataset_test 都是 optDataset 的实例
loader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
loader_test  = DataLoader(dataset_test,  batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

loader_train, loader_test = get_data_loaders(dataset_train, dataset_test)

loss_log=[]
loss_log_regret=[]

In [ ]:
from train import trainModel
from config import NUM_EPOCHS, BATCH_SIZE, LR, LSTM_SAVE_DIR
import os
os.makedirs(LSTM_SAVE_DIR, exist_ok=True)   
import pyepo
import torch 

set_seed(42)

spop = pyepo.func.SPOPlus(model)
loss_log, loss_log_regret = trainModel(
    lstm, 
    loss_func=spop, 
    method_name="spo+",
    loader_train=loader_train,
    loader_test=loader_test,
    market_neutral_model=model,
    params_testing=params_testing,
    loss_log=loss_log, 
    loss_log_regret=loss_log_regret,
    num_epochs=NUM_EPOCHS,  # Increased for better convergence
    lr=LR,
    initial=False,# Adjusted learning rate,
    scaler= False,
)

lstm_save_path = os.path.join(LSTM_SAVE_DIR, "trial_7_assets.pt")
torch.save({
    'model_state_dict': lstm.state_dict(),
    'loss_log': loss_log,
    'loss_log_regret': loss_log_regret
}, lstm_save_path)

print(f"模型参数已保存到 {lstm_save_path}")

Seed set to 42
Num of cores: 1


Epoch 1/3:  76%|███████▌  | 9962/13101 [01:25<00:27, 115.14it/s, loss=0.0000, batch time=0.0063s] 

In [ ]:
lstm_save_path = os.path.join(LSTM_SAVE_DIR, "trial_7_assets.pt")
torch.save({
    'model_state_dict': lstm.state_dict(),
    'loss_log': loss_log,
    'loss_log_regret': loss_log_regret
}, lstm_save_path)

print(f"模型参数已保存到 {lstm_save_path}")

In [ ]:
from train import trainModel
from config import NUM_EPOCHS, BATCH_SIZE, LR, LSTM_SAVE_DIR
import os
os.makedirs(LSTM_SAVE_DIR, exist_ok=True)   
import pyepo
import torch 

for i in range(100):

    print( f"第{i+1}次训练") 

    set_seed(42)
    
    spop = pyepo.func.SPOPlus(model)
    loss_log, loss_log_regret = trainModel(
        lstm, 
        loss_func=spop, 
        method_name="spo+",
        loader_train=loader_train,
        loader_test=loader_test,
        market_neutral_model=model,
        params_testing=params_testing,
        loss_log=loss_log, 
        loss_log_regret=loss_log_regret,
        num_epochs=NUM_EPOCHS,  # Increased for better convergence
        lr=LR,
        initial=False,# Adjusted learning rate,
        scaler= False
    )
    
    lstm_save_path = os.path.join(LSTM_SAVE_DIR, "trial_7_assets.pt")
    torch.save({
        'model_state_dict': lstm.state_dict(),
        'loss_log': loss_log,
        'loss_log_regret': loss_log_regret
    }, lstm_save_path)
    
    print(f"模型参数已保存到 {lstm_save_path}")

In [ ]:
#############################################################################
# VISUALIZATION
#############################################################################
from matplotlib import pyplot as plt
import numpy as np


def visLearningCurve(loss_log, loss_log_regret):
    """Enhanced visualization with smoother curves and more information"""
    # Create figure and subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4))
    
    # Plot training loss with smoothing for readability
    n_points = len(loss_log)
    
    # Apply smoothing for large datasets
    if n_points > 100:
        window_size = max(10, n_points // 50)
        smoothed_loss = np.convolve(loss_log, np.ones(window_size)/window_size, mode='valid')
        x_axis = np.arange(len(smoothed_loss))
        ax1.plot(x_axis, smoothed_loss, color="c", lw=2, label=f"Smoothed (window={window_size})")
        # Also plot the raw data with transparency
        ax1.plot(loss_log, color="c", lw=0.5, alpha=0.3, label="Raw")
        ax1.legend()
    else:
        # For smaller datasets, just plot the raw data
        ax1.plot(loss_log, color="c", lw=2)
    
    ax1.tick_params(axis="both", which="major", labelsize=12)
    ax1.set_xlabel("Iterations", fontsize=16)
    ax1.set_ylabel("Loss", fontsize=16)
    ax1.set_title("Learning Curve on Training Set", fontsize=16)
    ax1.grid(True, linestyle='--', alpha=0.7)
    
    # Draw plot for regret on test set
    epochs = np.arange(len(loss_log_regret))
    ax2.plot(epochs, [r*100 for r in loss_log_regret], 
             color="royalblue", marker='o', ls="-", alpha=0.8, lw=2)
    
    ax2.tick_params(axis="both", which="major", labelsize=12)
    ax2.set_ylim(0, max(50, max([r*100 for r in loss_log_regret])*1.1))  # Dynamic y-limit
    ax2.set_xlabel("Epochs", fontsize=16)
    ax2.set_ylabel("Regret (%)", fontsize=16)
    ax2.set_title("Learning Curve on Test Set", fontsize=16)
    ax2.grid(True, linestyle='--', alpha=0.7)
    
    # Add values to points
    for i, r in enumerate(loss_log_regret):
        ax2.annotate(f"{r*100:.2f}%", 
                     (i, r*100),
                     textcoords="offset points", 
                     xytext=(0,10), 
                     ha='center')
    
    plt.tight_layout()
    # plt.savefig('learning_curves.png', dpi=300, bbox_inches='tight')
    # print("Saved learning curves to 'learning_curves.png'")
    plt.show()

print("\nVisualizing learning curves...")
visLearningCurve(loss_log, loss_log_regret)

### 回测

In [ ]:
from train import trainModel
from config import NUM_EPOCHS, BATCH_SIZE, LR, LSTM_SAVE_DIR
import pyepo
import torch 
import os

lstm_save_path = os.path.join(LSTM_SAVE_DIR, "trial_7_assets.pt")

lstm_eval = TwoLayerLSTM(k=K, hidden_dim=HIDDEN_DIM, lstm_hidden_dim=LSTM_HIDDEN_DIM, dropout_rate=DROPOUT_RATE).to(DEVICE)

"""

lstm_eval = TCNModel(k=K, hidden_dim=HIDDEN_DIM, num_layers=4, dropout_rate=DROPOUT_RATE).to(DEVICE)

"""

checkpoint = torch.load(lstm_save_path, map_location=DEVICE, weights_only=False)
lstm_eval.load_state_dict(checkpoint['model_state_dict'])
print("Model architecture after training:")
for name, param in lstm_eval.named_parameters():
    print(f"{name:30s} | Shape: {param.data.shape} | Parameters: {param.numel()} | require_grad: {param.requires_grad}")
    print(f"Mean Value: {param.data.mean()} | Grad: {param.grad}")
    print("-" * 80)

In [ ]:
import numpy as np
# set lstm to evaluation mode
lstm_eval.eval()

pred_returns = [] # predicted returns
with torch.no_grad():
    for x, _, _, _ in loader_test:
        # 1. 预测 returns
        x = x.to(DEVICE)                      # (batch_size, N, lookback, k)
        pred_return = lstm_eval(x)                   # (batch_size, N)
        pred_return = pred_return.cpu().numpy()             # 转成ndarray
        pred_returns.append(pred_return)

pred_returns = np.vstack(pred_returns) # (T, N)
print(f"预测的收益率矩阵形状是: {pred_returns.shape}")

In [ ]:
print("标准化前的统计:")
print(f"  全局均值: {np.mean(pred_returns):.6f}")
print(f"  正数比例: {(pred_returns > 0).mean()*100:.1f}%")

# 对每个时间步进行横截面标准化
mean_cross_section = np.mean(pred_returns, axis=1, keepdims=True)  # (T, 1)
std_cross_section = np.std(pred_returns, axis=1, keepdims=True)    # (T, 1)
pred_returns_standardized = (pred_returns - mean_cross_section) / (std_cross_section + 1e-8)

print("标准化后的统计:")
print(f"  全局均值: {np.mean(pred_returns_standardized):.6f}")
print(f"  正数比例: {(pred_returns_standardized > 0).mean()*100:.1f}%")

In [ ]:
feats_test = np.load("test_data/features_crypto_data.npy")
print(f"测试集feature的形状是: {feats_test.shape}")

In [ ]:
### 优化器
from config import PRECOMPUTE_BATCH_SIZE, LOOKBACK, PADDING_METHOD
from batch_runner import process_and_combine_shared


#dataset_dict_test = process_and_combine_shared(feats_test, pred_returns, batch_size=PRECOMPUTE_BATCH_SIZE, **params)
from test_regret import sequential_solutions

positions=sequential_solutions(predmodel=lstm_eval,params_testing=params_testing, dataloader=loader_test)

In [ ]:
# 测试集仓位
#positions = dataset_dict_test["sols"]
positions=np.array(positions)
print(f"测试集仓位的形状是: {positions.shape}")

In [ ]:
pnl_series = []
for i in range(len(positions)):
    pnl = np.nansum(positions[i] * test_dataset_dict["costs"][i])
    pnl_series.append(pnl)
print(f"pnl series的长度是: {len(pnl_series)}")

### Visualize Cumulative Pnl

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_cum_pnl_series(pnl_series, title='Cumulative PnL', figsize=(10, 6)):
    """
    绘制累计 PnL 曲线。

    参数
    -----
    pnl_series : pandas.Series
        每期的 PnL 序列，索引为日期或时间点。
    title : str, optional
        图表标题，默认 'Cumulative PnL'。
    figsize : tuple, optional
        图表大小，默认为 (10, 6)。
    """
    # 计算累计 PnL
    cum_pnl = pd.Series(pnl_series).cumsum()

    # 绘图
    plt.figure(figsize=figsize)
    plt.plot(np.arange(len(cum_pnl)), cum_pnl.values, linewidth=2)
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Cumulative PnL')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_cum_pnl_series(pnl_series)


In [ ]:
pred_returns[0]

In [ ]:
positions[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 智能检测你的数据格式
def smart_pnl_plot(pred_returns):
    
    # 转换为numpy数组
    if hasattr(pred_returns, 'cpu'):
        data = pred_returns.cpu().numpy()
    else:
        data = np.array(pred_returns)
    
    print(f"数据形状: {data.shape}")
    
    if data.ndim == 2:
        # 判断是 (时间, 资产) 还是 (资产, 时间)
        if data.shape[0] > data.shape[1]:
            # 假设是 (时间, 资产)
            print("格式: (时间, 资产)")
            avg_returns = np.mean(data, axis=1)  # 每个时间步的平均收益
        else:
            # 假设是 (资产, 时间)
            print("格式: (资产, 时间)")
            avg_returns = np.mean(data, axis=0)  # 每个时间步的平均收益
    else:
        print(f"数据维度: {data.ndim}")
        return None
    
    # 计算累计PnL
    pnl = np.cumsum(avg_returns)
    
    # 绘图
    plt.figure(figsize=(12, 6))
    plt.plot(pnl, linewidth=2)
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    plt.title('Portfolio PnL')
    plt.xlabel('Time')
    plt.ylabel('Cumulative PnL')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"最终PnL: {pnl[-1]:.6f}")
    return pnl



In [ ]:
# 使用你的数据
smart_pnl_plot(pred_returns_standardized)

In [ ]:
# 使用你的数据
smart_pnl_plot(test_dataset_dict["costs"])